In [1]:
import requests
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from IPython.display import display


In [2]:

def coingecko_price_history(coin, from_timestamp, to_timestamp):
    ### Input params:
    # coin - the full name of the coin like "bitcoin" or "ethereum"
    # from_timestamp = int(datetime(2021, 1, 1).timestamp())
    # to_timestamp same like from_timestamp

    url = "https://api.coingecko.com/api/v3/coins/" + coin + "/market_chart/range"
    parameters = {
        'vs_currency': 'usd',
        'from': from_timestamp,
        'to': to_timestamp
    }
    
    response = requests.get(url, params=parameters)
    data = response.json()

    prices_df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    volumes_df = pd.DataFrame(data['total_volumes'], columns=['timestamp', 'volume'])
    market_caps_df = pd.DataFrame(data['market_caps'], columns=['timestamp', 'market_cap'])

    # Merge dataframes based on timestamp
    merged_df = prices_df.merge(volumes_df, on='timestamp', how='outer')
    merged_df = merged_df.merge(market_caps_df, on='timestamp', how='outer')
    
    # Convert timestamp to datetime and set it as the index
    merged_df['date'] = pd.to_datetime(merged_df['timestamp'], unit='ms')
    merged_df.set_index('date', inplace=True)
    merged_df.drop(columns=['timestamp'], inplace=True)

    return merged_df.sort_index()  # Sort by date to ensure it's in the right order


In [3]:
# Convert human-readable date to timestamp
start_date = datetime(2002, 1, 1)
end_date = datetime(2023, 8, 13)
from_timestamp = int(start_date.timestamp())
to_timestamp = int(end_date.timestamp())

df = coingecko_price_history('bitcoin', from_timestamp, to_timestamp)

In [4]:
df['smoothed_price'] = df['price'].rolling(window=7).mean()
df['111D_MA']        = df['smoothed_price'].rolling(window=111).mean()
df['350D_MA_x2']     = df['smoothed_price'].rolling(window=350).mean() * 2.0
df['Crossover_Up']   = (df['111D_MA'] > df['350D_MA_x2']) & (df['111D_MA'].shift(1) <= df['350D_MA_x2'])
crossover_up_dates   = df[df['Crossover_Up']].index

In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['price'], mode='lines', name='BTC Price'))
fig.add_trace(go.Scatter(x=df.index, y=df['111D_MA'], mode='lines', name='111DMA'))
fig.add_trace(go.Scatter(x=df.index, y=df['350D_MA_x2'], mode='lines', name='350DMA x 2'))

for date in crossover_up_dates:
    fig.add_shape(
        type='line',
        x0=date, y0=0, x1=date, y1=1,
        yref='paper',
        line=dict(color='Red', dash='dot')
    )

fig.update_layout(title='BTC Price Pi Cycle Top Indicator', xaxis_title='Date', yaxis_title='Price (in USD)', yaxis_type='log', height = 1000)
display(fig)
#df[['price', '111D_MA', '350D_MA_x2']].iplot(title="BTC Pi Cycle Top", theme="solar")
